In [1]:
import pandas as pd
import os, pathlib
import numpy as np
from import_clean import *
from calculate import *
from itertools import combinations

In [ ]:
all_data = cleanup_all(import_all_data("utla"), remove_geography_code=False)

In [ ]:
keys = list(all_data.keys())
for k1, k2 in combinations(keys, 2):
    dataframes = [all_data[k1], all_data[k2]]
    columns = [[column for column in df.columns if column != "geography code" and "Total" not in column] for df in dataframes]
    totals = [[column for column in df.columns if "Total" in column][0] for df in dataframes]
    df_merged = dataframes[0].merge(dataframes[1], on="geography code")
    for column, total in zip(columns, totals):
        df_merged[column] = df_merged[column].divide(df_merged[total], axis=0)
    
